# GrobidClient Interactive Playground

This notebook demonstrates how to use `grobid-client-python` to process a PDF and extract its fulltext XML output.

In [7]:
# Import Required Libraries
import sys
from grobid_client.grobid_client import GrobidClient

In [8]:
# Initialize GrobidClient
client = GrobidClient(config_path="/home/jakub/Projects/paper_testing/config.json")  # use Docker container URL

GROBID server is up and running


In [ ]:
# Process PDF to Extract Fulltext
pdf_path = "/home/jakub/Projects/paper_testing/documents_to_test/pdfs"  # Change this to your PDF file
output_dir = "output"  # Output directory for results

result = client.process("processFulltextDocument", pdf_path, n=20, consolidate_citations=True, tei_coordinates=True, consolidate_header=True, include_raw_affiliations=True, include_raw_citations=True, verbose=True, force=True, output=output_dir)

kelders-et-al-2024.pdf
validate_references_fake_paper.pdf
2 files to process in current batch
Adding /home/jakub/Projects/paper_testing/documents_to_test/pdfs/kelders-et-al-2024.pdf to the queue.
Adding /home/jakub/Projects/paper_testing/documents_to_test/pdfs/fake/validate_references_fake_paper.pdf to the queue.


KeyboardInterrupt: 

In [4]:
# Display Output Path
print(f"GROBID output saved to: {result}")

GROBID output saved to: None
